In [13]:
import pandas as pd
data_url= "http://www-stat.wharton.upenn.edu/~waterman/DataSets/uva.txt"
dataframe = pd.read_table(data_url)

In [14]:
dataframe.pop('who') # dataframe = dataframe.ix[:, 1:]
dataframe.pop('Country')
dataframe.pop('Years on Internet')

dataframe.dtypes

for col in ["Gender", 'Household Income', 'Sexual Preference', 'Education Attainment', 'Major Occupation', "Marital Status"]:
    dataframe[col] = dataframe[col].astype('category')

dataframe_modified = pd.get_dummies(dataframe)
dataframe_modified['Age'][pd.isnull(dataframe_modified['Age'])] = dataframe_modified['Age'].mean()

x_data = dataframe_modified.ix[:, 1:].values
y_data = dataframe_modified.ix[:, 0].reshape(-1,1)

/opt/conda/envs/ml_python34/lib/python3.4/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
from sklearn import preprocessing # Min-Max Standardzation

min_max_scaler = preprocessing.MinMaxScaler()
x_data[:, 0] = min_max_scaler.fit_transform(x_data[:, 0])

x_data[:5,0]

/opt/conda/envs/ml_python34/lib/python3.4/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/opt/conda/envs/ml_python34/lib/python3.4/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


array([ 0.65333333,  0.45333333,  0.58666667,  0.22666667,  0.2       ])

In [16]:
y_data.shape, x_data.shape

((19583, 1), (19583, 37))

In [17]:
import tensorflow as tf

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# Deep network configuration.: Use more layers. 
W1 = tf.Variable(tf.random_uniform( [37,100], -1.0, 1.0))
W2 = tf.Variable(tf.random_uniform( [100,30], -1.0, 1.0))
W3 = tf.Variable(tf.random_uniform( [30,1], -1.0, 1.0))

b1 = tf.Variable(tf.zeros([100]), name="Bias1")
b2 = tf.Variable(tf.zeros([30]), name="Bias2")
b3 = tf.Variable(tf.zeros([1]), name="Bias3")

# Hypotheses 
L2 =  tf.sigmoid(tf.matmul(X,W1)+b1)
L3 =  tf.sigmoid(tf.matmul(L2,W2)+b2)
hypothesis = tf.sigmoid( tf.matmul(L3,W3) + b3)

# Cost function 
cost = -tf.reduce_mean( Y*tf.log(hypothesis)+(1-Y)* tf.log(1.-hypothesis) )

# Minimize cost.
a = tf.Variable(0.1)
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# Initializa all variables.
init = tf.initialize_all_variables()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(20001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        
        if step % 1000 == 0:
            print(
                step, 
                sess.run(cost, feed_dict={X:x_data, Y:y_data}), 
                sess.run(W1),
                sess.run(W2)
            )
    
    # Test model
    correct_prediction = tf.equal( tf.floor(hypothesis+0.5), Y)
    accuracy = tf.reduce_mean(tf.cast( correct_prediction, "float" ) )
    
    # Check accuracy
    print( sess.run( [hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], 
                   feed_dict={X:x_data, Y:y_data}) )
    print( "Accuracy:", accuracy.eval({X:x_data, Y:y_data}) )

0 0.806677 [[-0.63897491  0.1070373   0.57480675 ...,  0.57027483 -0.76128727
  -0.83125675]
 [-0.16075568  0.18166807  0.05362928 ...,  0.6178987   0.38760296
  -0.62282693]
 [-0.90363795  0.68639874 -0.47901031 ..., -0.15704013  0.48662278
  -0.97261232]
 ..., 
 [ 0.25934157  0.01314267 -0.03699095 ...,  0.99785006 -0.71375555
   0.16265735]
 [-0.62938917  0.53598034 -0.011912   ..., -0.25080475 -0.12241752
  -0.87665772]
 [-0.48885116 -0.37375858  0.6904375  ..., -0.57496524  0.57303584
   0.57382673]] [[ 0.41003311 -0.52294481  0.36437568 ...,  0.46257812  0.94404638
   0.14516476]
 [-0.99016625 -0.7082085  -0.6250686  ..., -0.71673638 -0.54643327
  -0.02295076]
 [ 0.7567032  -0.6090433  -0.40126342 ...,  0.63943177  0.21582505
   0.66657019]
 ..., 
 [-0.26798204 -0.37551108 -0.86643088 ..., -0.29967344  0.11301666
   0.79574388]
 [ 0.28559226 -0.96297765  0.17727098 ..., -0.41888759 -0.38722551
   0.09849125]
 [ 0.55204356 -0.82328451 -0.98364663 ...,  0.90175575  0.05796745
  -0.

KeyboardInterrupt: 